<img src="../.images/logosnb.png" alt="Banner" style="width: 800px;"/>

<div style='color: #690027;' markdown="1">
    <h1>DE FUNDAMENTEN VAN EEN DIEP NEURAAL NETWERK VOOR BEELDHERKENNING</h1> 
</div>

<div class="alert alert-box alert-success">
In deze notebook zal je stap voor stap een diep neuraal netwerk opbouwen dat in staat is een onderscheid te maken tussen een afbeelding met een stoma en een afbeelding zonder stoma. Men noemt dat een classificatieprobleem: een afbeelding wordt geclassificeerd als 'stoma' of 'geen stoma'.
</div>

Voer onderstaande code-cel uit om van de functies in deze notebook gebruik te kunnen maken.

In [ ]:
import imp
with open('../.scripts/diep_neuraal_netwerk.py', 'rb') as fp:
    diep_neuraal_netwerk = imp.load_module('.scripts', fp, '../.scripts/diep_neuraal_netwerk.py', ('.py', 'rb', imp.PY_SOURCE))

<div style='color: #690027;' markdown="1">
    <h2>1. De data</h2> 
</div>

Een diep neuraal netwerk leert een invoer af te beelden op een uitvoer door gelabelde data te verwerken. Deze data bestaan uit invoer en de daarbij horende verwachte uitvoer van het model. Voor het stomataprobleem zijn dat microfoto's van delen van bladeren van verschillende soorten planten uit het tropisch regenwoud. <br>
Alle foto's hebben een formaat van 120 op 120 pixels. Deze foto's zijn opgedeeld in twee klassen die de uitvoer van het netwerk voorstellen. De foto's met een stoma in het midden behoren tot de klasse 'Stoma', de foto's zonder een stoma, met een gedeeltelijke stoma of met een stoma die niet in het midden van de afbeelding staat, behoren tot de klasse 'Geen stoma'. <br>
<b>Er is experimenteel bepaald dat er om een zo goed mogelijk resultaat te bekomen 6 keer meer foto's zonder een stoma nodig zijn dan foto's  met een stoma. </b>Dit komt omdat er meer variëteit zit in de afbeeldingen zonder stoma waardoor er dus ook meer voorbeelden nodig zijn.<br> <br>De volgende afbeelding geeft een voorbeeld van de beschikbare invoer en de daarbij horende uitvoer.

<img src="../.images/IntroductieDeepLearning/training_data.jpg"/>

De beschikbare data worden het best opgedeeld in 3 sets:
<ul>
    <li><b>Trainingset</b>: Dit is de data die gebruikt worden om het model te trainen.<br>76 740 foto's met label 'Geen stoma' + 12 790 foto's met label 'Stoma' = 89 530 trainingsafbeeldingen</li>
    <li><b>Validatieset</b>: Deze data worden gebruikt om te kijken hoe goed het netwerk presteert op data die het nog niet gezien heeft. Op basis van deze data wordt het netwerk bijgeschaafd om betere resultaten te bekomen.<br>28 866 foto's met label 'Geen stoma' + 4 811 foto's met label 'Stoma'  = 33 677 valideringsafbeeldingen</li>
    <li><b>Testset</b>: Na het trainen met de trainingset en bijschaven van het netwerk aan de hand van de validatieset wordt het netwerk nog één keer geëvalueerd met de testset om de eindscore te berekenen. <br>55 182 foto's met label 'Geen stoma' + 9 197 foto's met label 'Stoma' = 64 379 test afbeeldingen</li>
</ul>

Het lijkt misschien overbodig om een testset te hebben om het netwerk nog een laatste keer te evalueren omdat men al gebruik maakt van de validatieset om te kijken hoe goed het netwerk presteert. Toch is dit noodzakelijk, de resultaten van de validatieset gebruikt men immers om het netwerk aan te passen opdat het beter presteert met deze validatieset. Op deze manier zoekt men het beste netwerk voor een specifieke validatieset en is de uiteindelijke scoren op deze set ongeschikt om in te schatten hoe het netwerk presteert op nieuwe data. De beoordeeling op een testset die het netwerk nog nooit heeft gezien is hiervoor meer geschikt.

<div style='color: #690027;' markdown="1">
    <h2>2. De netwerk architectuur</h2> 
</div>

Een diep neuraal netwerk bestaat uit verschillende opeenvolgende lagen die de invoer omzetten in de uitvoer. <br> De volgende afbeelding stelt een basis netwerk voor. 
<img src="../.images/IntroductieDeepLearning/vb_netwerk.jpg"/>
De volgende subparagrafen beschrijven elk een deel van het netwerk. Het cijfer 1 op de afbeelding staat dus voor paragraaf 2.1, het cijfer 2 voor paragraaf 2.2, enz.

<div style='color: #690027;' markdown="1">
    <h2>2.1 Invoer</h2> 
</div>

Zoals eerder vermeld is de invoer van het netwerk een microfoto van een blad waarop wel of juist geen stoma te zien is en heeft deze foto een formaat van 120 op 120 pixels. De foto zal worden voorgesteld als een 3D-tensor met dimensie $3x120x120$ zodat hij verwerkt kan worden door het netwerk.

<div class="alert alert-block alert-warning"> 
Voor uitleg over tensoren, kan je terecht in de notebooks 'Tensoren' en 'Tensoren en RGB'.
</div>

<div style='color: #690027;' markdown="1">
    <h2>2.2 Convolutionele lagen</h2> 
</div>

Om relevante patronen te ontdekken in de ingevoerde afbeelding (voorgesteld door een tensor) worden convolutionele lagen gebruikt. Een convolutionele laag zal een tweede 3D-tensor, een (<b>filter</b>), met een zelfgekozen x- en y-dimensie en dezelfde z-dimensie als de invoer op de afbeelding leggen en een wiskundige berekening, een convolutie, uitvoeren die één getal teruggeeft. Hierna zal deze filter een zelfgekozen aantal pixels opschuiven en wordt het volgende getal berekend, hierdoor verkrijgt men een matrix die informatie bevat over waar het patroon van de filter voorkomt in de invoer. De volgende afbeelding toont deze operatie op een afbeelding van 8 op 8 pixels met diepte 3 (RGB) en met als filter een 3D-tensor met dimensie $3x3x3$ die steeds één positie opschuift.

<img src="../.images/IntroductieDeepLearning/conv_operation.jpg" width="500"/>

Een convolutionele laag zal meestal niet één maar meerdere filters gebruiken om verschillende patronen te herkennen. Hierdoor krijgt men per filter die over de afbeelding glijdt een matrix met dezelfde dimensie. Door deze uitvoeren te combineren, bekomt men opnieuw een 3D-tensor waarbij de z-dimensie gelijk is aan het aantal filters. Deze tensor wordt een <b>feature map</b> genoemd. De waarden van deze filters worden aangepast tijdens het trainen van het model om relevante patronen te herkennen. 

In de afbeelding van het basisnetwerk stelt het getal onder de convolutionele laag (hier 32) het aantal filters en dus de z-dimensie van de uitvoertensor van de convolutionele laag voor.

<img src="../.images/IntroductieDeepLearning/conv_layer_2.jpg"/>

Het netwerk dat je samenstelt voor de classificatie van stomata zal filters gebruiken met lengte en breedte 3 en deze filters zullen steeds 1 positie per keer opschuiven.

<div class="alert alert-block alert-warning"> 
Hoe verrassend het effect van convoluties is, zie je in de notebook 'Convolutie'. Om te weten hoe het werkt, bekijk je de notebook 'Convolutie: de bewerking'.
</div>

<div style='color: #690027;' markdown="1">
    <h2>2.3 Max-pooling</h2> 
</div>

De convolutionele lagen worden afgewisseld met max-pooling operaties, deze operaties houden uit een venster met zelfgekozen afmetingen enkel de grootste waarde over. Die grootste waarde geeft immers aan waar de filter het meest aanwezig was in de invoer. <br>
De volgende afbeelding geeft een voorbeeld van de max-pooling operatie.

<img src="../.images/IntroductieDeepLearning/max_pooling.jpg"/>

Het doel van max-pooling operaties is tweedelig. Enerzijds wil men de grootte van de uitvoer van de convolutionele lagen verkleinen; waarom dit nodig is wordt beschreven in de volgende paragraaf. Anderzijds wil men meer informatie bekomen over een groter deel van de ingevoerde afbeelding. <br>
Stel je even voor dat de max-pooling operaties er niet zouden zijn. Na twee opeenvolgende convolutionele lagen zou een veld met lengte en breedte 1 dan slechts informatie bevatten over een veld met lengte en breedte 5 van de ingevoerde afbeelding. Dit is vaak niet genoeg om belangrijke kenmerken van de afbeelding te herkennen.  Zou je zelf een afbeelding herkennen aan een aantal pixels?<br> 
Volgende afbeelding toont wat er gebeurt met twee convolutionele lagen (met steeds maar 1 filter) zonder max-pooling operaties. Het rode veld in de laatste feature map bevat enkel informatie uit het rode veld uit de ingevoerde afbeelding.

<img src="../.images/IntroductieDeepLearning/conv_no_max_pooling.jpg"/>

In het KIKS-netwerk gebruiken we max-pooling in velden met een lengte en breedte van 2, hierdoor wordt de grootte van de feature maps in vier gedeeld door een max-pooling operatie.

<div style='color: #690027;' markdown="1">
    <h2>2.4 Feedforward lagen</h2> 
</div>

Om de effectieve classificatie van de afbeelding uit te voeren gebruiken men feedforward lagen. <br>
Deze lagen bestaan uit neuronen ([StomataZonSchaduw](../IntroductieMachineLearning/StomataZonSchaduw.ipynb)) waarbij elk neuron is verbonden met elk ander neuron van de volgende feedforward laag. Al deze verbindingen tussen de neuronen van de opeenvolgende lagen hebben een bepaalde waarde. Men kan het geheel vergelijken met een functie die de invoer van een laag omzet naar de uitvoer van die laag en waarbij de neuronen de variabelen zijn van de functie en de verbindingen de coëfficiënten. Deze coëfficiënten worden ook wel de <b>gewichten</b> van het netwerk genoemd. <br>
Een netwerk <em>leert</em> door deze gewichten aan te passen afhankelijk van de trainingdata. Hoe meer neuronen, hoe meer informatie het netwerk kan opslaan. Teveel neuronen is echter ook niet altijd goed (zie notebook [Overfitting](../IntroductieDeepLearning/Overfitting.ipynb)).<br>
De volgende afbeelding toont een netwerk dat alleen uit feedforward lagen bestaat: de cirkels zijn de neuronen, de verbindingen zijn de gewichten.

<img src="../.images/IntroductieDeepLearning/ffn.jpg"/>

Tot nu toe is de ingevoerde afbeelding door de convolutionele lagen en de max-pooling operaties omgezet in een 3D-tensor die informatie bevat over verschillende relevante patronen in die afbeelding. <br>
Vooraleer deze data als invoer van de feedforward lagen kunnen dienen moet de 3D-tensor worden omgezet in een rijmatrix, dit doen we met een <b>flatten</b>-operatie. Volgende afbeelding toont hoe de flatten-operatie een feature map met dimensie $2x3x3$ omzet in een rijmatrix om als invoer te dienen voor de feedforward lagen.

<img src="../.images/IntroductieDeepLearning/flatten.jpg"/>

Nu is ook meteen duidelijk waarom de uitvoer van de convolutionele lagen niet te groot mag zijn. Neem bijvoorbeeld een flatten-operatie op een feature map met dimensie $64x100x100$, dit geeft 64 0000 verschillende invoeren voor de feedforward lagen. Als de eerste feedforward laag dan nog eens 64 neuronen bevat, zijn er tussen deze twee lagen alleen al bijna 41 miljoen gewichten die het netwerk moet leren.

In de voorstelling van het diep neuraal netwerk wordt een feedforward laag voorgesteld als een paars balkje. Het getal onder het balkje stelt het aantal neuronen in deze laag voor.

<div style='color: #690027;' markdown="1">
    <h2>2.5 Uitvoer</h2> 
</div>

De laatste laag bestaat uit slechts één neuron en geeft een getal tussen 0 en 1 terug. Dit is de voorspelling van het netwerk waarbij 0 staat voor 'Geen stoma' en 1 voor 'Stoma'. Hoe dichter de uitvoer bij 1 ligt hoe zekerder het netwerk is dat er op de ingevoerde afbeelding een stoma te zien is.

<div style='color: #690027;' markdown="1">
    <h2>2.6. Kies je netwerk architectuur</h2> 
</div>

Nu je de verschillende onderdelen van een diep neuraal netwerk begrijpt kan je zelf je netwerk samenstellen om de stomataclassificatie uit te voeren. Voer onderstaande code-cel uit om de parameters van het netwerk te kiezen.

In [ ]:
diep_neuraal_netwerk.kies_netwerk_parameters()

Om je gekozen netwerk te visualiseren voer je volgende instructie uit. Ben je niet tevreden of wil je andere zaken uitproberen, verander dan gerust de parameters hierboven en voer de instructie opnieuw uit.

In [ ]:
diep_neuraal_netwerk.toon_netwerk()

<div style='color: #690027;' markdown="1">
    <h2>3. Het netwerk trainen</h2> 
</div>

Nu de netwerkarchitectuur gekozen is moet dit netwerk getraind worden. Vooraleer je dit kunt doen moeten er opnieuw een aantal keuzes worden gemaakt.

<div style='color: #690027;' markdown="1">
    <h2>3.1 Epochs</h2> 
</div>

Je kan ervoor kiezen hoeveel keer de volledige training data verwerkt worden, dit wordt het aantal <b>epochs</b> genoemd. Vaak moet je verschillende epochs proberen om het beste resultaat te bekomen. Ons netwerk is steeds getraind met 50 epochs.

<div style='color: #690027;' markdown="1">
    <h2>3.2 Loss functie</h2> 
</div>

De loss functie is een functie die beschrijft hoe goed het netwerk presteert met de huidige gewichten. Hoe lager de waarde van de loss functie hoe dichter de huidige uitvoer van het netwerk bij de gewenste uitvoer ligt. Het netwerk zal een aantal afbeeldingen samen nemen (<b>een batch</b>) en kijkt dan in welke mate deze juist geclassificeerd werden. In ons netwerk is er gekozen voor de loss functie binary crossentropy. Deze loss functie wordt vaak gebruikt bij classificatie problemen met 2 klassen (vandaar de 'binary'). We gaan hier niet in detail hoe deze loss functie juist werkt.

<div style='color: #690027;' markdown="1">
    <h2>3.3 Optimizer</h2> 
</div>

Een andere belangrijke keuze bij het trainen van een diep neuraal netwerk is de <b>optimizer</b>. Deze bepaalt op welke manier het netwerk leert. In ons netwerk is er gekozen om de <b>stochastic gradient descent (SGD)</b> optimizer te gebruiken omdat deze het minst complex is. Zoals de naam al doet vermoeden maakt SGD gebruik van [gradient descent](../IntroductieDeepLearning/GradientDescent.ipynb), de afgeleide van de loss functie wordt berekend om zo een minimum hiervoor te vinden. De 'stochastic' betekent dat de afgeleide van de loss niet wordt berekent met de volledige trainingset maar met slechts een willekeurig deel hiervan. Dit gaat een stuk sneller dan wanneer de afgeleide van de loss wordt berekend met de volledige trainingset.

<div style='color: #690027;' markdown="1">
    <h2>3.4 Learning rate</h2> 
</div>

De learning rate bepaalt hoe groot de stap is om in het minimum te geraken, een kleine learning rate zal ervoor zorgen dat het netwerk traag leert en met een te grote learning rate zal het netwerk het minimum niet vinden. Volgende afbeelding toont hoe gradient descent werkt in 2 dimensies. Bij een diep neuraal netwerk zijn er enorm veel gewichten en dus ook enorm veel dimensies wat veel moeilijker voor te stellen is door een figuur.

<img src="../.images/IntroductieDeepLearning/gradient_descent.jpg" width="500"/>

Voer onderstaande code-cel uit om de learning rate te kiezen waarmee het netwerk wordt getraind.

In [ ]:
diep_neuraal_netwerk.kies_training_parameters()

<div style='color: #690027;' markdown="1">
    <h2>4. Resultaten</h2> 
</div>

Normaal gezien duurt het even om een netwerk te trainen. De netwerken die in dit voorbeeld gemaakt kunnen worden, zijn echter op voorhand getraind en de resultaten zijn opgeslagen in een databank. Op deze manier kan je dus onmiddellijk kijken naar de prestaties van het gekozen netwerk. 

Er bestaan verschillende manieren om een netwerk te beoordelen. Enerzijds kan je kijken naar de waarde van de loss functie, anderzijds kan je ook kijken naar de <b>accuracy</b> of nauwkeurigheid van het netwerk, dit is het percentage van de samples waarvoor het netwerk de voorspelling juist heeft. Deze waarden kan je tijdens het trainen per epoch berekenen voor de training data en de validatie data. Na het trainen kan je deze waarden ook nog 1 maal berekenen op de testdata om de finale beoordeling van het netwerk te bekomen.

Door volgende code-cel uit te voeren zie je de training en validatie accuracy en loss voor het netwerk. De grafieken geven per epoch de training en de validatie accuracy of loss weer.

In [ ]:
diep_neuraal_netwerk.toon_grafiek()

Vaak zal je zien dat de training loss daalt terwijl de validatie loss stijgt. Wanneer dit gebeurt, is het netwerk aan het <b>overfitten</b>. Dit wil zeggen dat het netwerk te veel details van de training data van buiten leert en dus niet meer goed generaliseert op data dat het nog nooit gezien heeft. Overfitting is een van de grootste problemen van een diep neuraal netwerk, daarom zal een volgende notebook ([Overfitting](../IntroductieDeepLearning/Overfitting.ipynb)) een aantal technieken uitleggen om overfitting tegen te gaan.

Het tegenovergestelde van overfitten is <b>underfitten</b>: dit wil zeggen dat het netwerk niet genoeg kan leren en dus slecht het relevante patroon in de data kan herkennen. Vaak is dit het geval bij een te simpel netwerk of een netwerk dat te kort werd getraind.

Volgende afbeelding toont hoe je underfitting en overfitting kan herkennen op een grafiek waarbij de loss waarden van de verschillende epochs weergegeven zijn. Wanneer de grafiek zou stoppen voor de meest linkse stippellijn, dan heb je een netwerk dat underfit. Wanneer de grafiek zou stoppen na de meest rechtse stippellijn, heb je een netwerk dat overfit.

<img src="../.images/IntroductieDeepLearning/underfitting_overfitting.jpg"/>

Een laatste geval is dat het netwerk niet leert. Om dit vast te stellen formuleer je een <b>baseline</b>. Wat dit is is makkelijk uit te leggen aan de hand van het KIKS-voorbeeld. Onze dataset bevat 6 keer zo veel afbeeldingen zonder stoma dan afbeeldingen met een stoma. In totaal is dus 6/7 (85.7%) van de trainingset, validatieset en testset een afbeelding zonder stoma. Wanneer het model dus alle afbeeldingen als 'geen stoma' aangeeft, zal er dus al een accuracy van 85.7% zijn. Dit is de baseline die het model moet overtreffen vooraleer je kunt zeggen dat het model iets heeft geleerd.

<div style='color: #690027;' markdown="1">
    <h2>4.1 Oefening</h2> 
</div>

Zoek door in "2.6. Kies je netwerk architectuur" en "3.4 Learning rate" de parameters aan te passen en in "4. Resultaten" te kijken naar de grafieken een model dat underfit, een model dat overfit en een model dat niet leert.

<div style='color: #690027;' markdown="1">
    <h2>4.2 Voorspellingen</h2> 
</div>

Als extra tonen we nog een aantal voorspellingen van het netwerk. De voorspelling is een getal tussen 0 en 1 dat aangeeft hoe zeker het netwerk is dat er een stoma gevonden werd. Hier is ook de <b>drempelwaarde</b> van belang. Deze drempelwaarde bepaalt voor welke waarden van de uitvoer we de invoer als een stoma beschouwen. Als we bijvoorbeeld 0.5 als drempelwaarde nemen zal alle uitvoer groter dan 0.5 als "Stoma" beschouwd worden en alle uitvoer kleiner dan 0.5 als "Geen stoma". 

Voer onderstaande code-cel uit om de voorspellingen te zien, speel met de drempelwaarde (thr: weergegeven in procent) en kijk of het netwerk of het netwerk de afbeelding juist classificeert (een groene rand) of fout classificeert (een rode rand).

In [ ]:
diep_neuraal_netwerk.toon_voorspellingen()

Er zijn ook afbeeldingen waar het model het moeilijk mee heeft. Wanneer het netwerk oordeelt dat op een afbeelding met een stoma geen stoma staat spreekt men van een <b>false negative</b>. Omgekeerd, wanneer het netwerk oordeelt dat er op een afbeelding zonder stoma wel een stoma staat spreekt men van een <b>false positive</b>. 

Voer onderstaande code-cel uit om enkele afbeeldingen te zien waarmee veel modellen het moeilijk hebben.

In [ ]:
diep_neuraal_netwerk.toon_slechte_voorspellingen()

Ben je tevreden over je netwerk, dan ga je het nog een laatste keer beoordelen aan de hand van de testset. Voeg volgende code-cel uit om het het uiteindelijke resultaat van je netwerk te zien.

In [ ]:
diep_neuraal_netwerk.toon_test_resultaten()

<img src="../.images/cclic.png" alt="Banner" align="left" style="width:80px;"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">ai op school</a>, van F. wyffels, A. Meheus, T. Neutens & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 